In [2]:
import numpy as np 
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt

In [3]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
#Preprocessing the data
X_train_full = X_train_full/255.0
X_test = X_test/255.0

X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_train = X_train.reshape(-1, 28*28)
X_valid = X_valid.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [5]:
#Building a Wide and Deep Model with Functional API

input_ = keras.layers.Input(shape=[28*28])

hidden1 = keras.layers.Dense(300, activation="relu")(input_)
hidden2 = keras.layers.Dense(100, activation="relu")(hidden1)

#Wide Path
concat = keras.layers.concatenate([input_, hidden2])

output = keras.layers.Dense(10)(concat)

model = keras.Model(inputs=[input_], outputs=[output])

E0000 00:00:1752514028.987222   10498 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1752514028.988649   10498 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
model.compile(loss="sparse_categorical_crossentropy",
     optimizer="sgd", 
     metrics=["accuracy"])

In [7]:
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20


/home/onlyahad/miniconda3/envs/ml_global/lib/python3.11/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 784))
  warnings.warn(msg)


1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1363 - loss: 7.7679 - val_accuracy: 0.1316 - val_loss: 4.8757
Epoch 2/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1442 - loss: 3.6404 - val_accuracy: 0.1936 - val_loss: 3.5881
Epoch 3/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.2003 - loss: 3.6181 - val_accuracy: 0.1940 - val_loss: 3.5383
Epoch 4/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.2060 - loss: 3.4998 - val_accuracy: 0.1938 - val_loss: 3.5435
Epoch 5/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.2019 - loss: 3.4228 - val_accuracy: 0.1652 - val_loss: 3.4801
Epoch 6/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.1830 - loss: 3.4122 - val_accuracy: 0.0980 - val_loss: 3.6528
Epoch 7/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.1010 - loss: 3.5923 - val_accuracy: 0.0980 - val_loss: 3.6418
Epoch 8/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.0991 - loss: 3.5877 - val_accurac

In [8]:
#Visualizing the model

keras.utils.plot_model(model, "wide_and_deep.png", show_shapes=True)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [10]:
model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0978 - loss: 6.6437


[6.781336784362793, 0.10000000149011612]

In [11]:
#SUBCLASSING 

class WideAndDeepModel(keras.Model):
    def __init__(self, hidden_units=100):
        super().__init__()
        self.hidden1 = keras.layers.Dense(300, activation="relu")
        self.hidden2 = keras.layers.Dense(hidden_units, activation="relu")
        self.output_layer = keras.layers.Dense(10)

    def call(self, inputs):
        hidden1 = self.hidden1(inputs)
        hidden2 = self.hidden2(hidden1)
        concat = tf.concat([inputs, hidden2], axis=1)
        return self.output_layer(concat)

In [12]:
model_sub = WideAndDeepModel()
model_sub.compile(loss="sparse_categorical_crossentropy",
                  optimizer="sgd",
                  metrics=["accuracy"])

history = model_sub.fit(X_train, y_train, epochs=20,
                        validation_data=(X_valid, y_valid))

Epoch 1/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.1118 - loss: 2.3403 - val_accuracy: 0.1112 - val_loss: 2.2380
Epoch 2/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0980 - loss: 2.2377 - val_accuracy: 0.1112 - val_loss: 2.2308
Epoch 3/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0988 - loss: 2.2275 - val_accuracy: 0.1112 - val_loss: 2.2304
Epoch 4/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.0989 - loss: 2.2313 - val_accuracy: 0.1112 - val_loss: 2.2297
Epoch 5/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.1011 - loss: 2.2355 - val_accuracy: 0.2134 - val_loss: 2.1636
Epoch 6/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.1963 - loss: 2.1308 - val_accuracy: 0.2136 - val_loss: 2.1781
Epoch 7/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.1920 - loss: 2.1862 - val_accuracy: 0.0976 - val_loss: 2.3026
Epoch 8/20
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.1003 - loss: 2.3026 - 

In [13]:
model_sub.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0993 - loss: 2.3026


[2.30259108543396, 0.10000000149011612]

In [16]:
model_sub.build(X_train.shape)
model_sub.summary()

Model: "wide_and_deep_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         8,850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 274,452 (1.05 MB)

 Trainable params: 274,450 (1.05 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)